In [ ]:
import pandas as pd

df = pd.read_excel("./input/rcr.xlsx", skiprows=12)
dfs = [df]

In [ ]:
import os
import re

for fp in os.listdir("./historic/"):
    df = pd.read_csv(f"./historic/{fp}")
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [ ]:
# df["CLASS"] = ""
# df["Form Factor"] = ""
# uk = df[df.Country=="United Kingdom"]
# uk_apple_surface = uk[uk["Sub Family"].isin(["Apple Other", "Surface Pro", "Surface Laptop"])]
# uk_other = uk[~uk["Sub Family"].isin(["Apple Other", "Surface Pro", "Surface Laptop"])]

# figs = df[df.Country!="United Kingdom"]
# figs_windows = figs[figs["OS Ecosystem"]=="Windows"]

In [ ]:
# df["CLASS"] = df["Sub Family"]
# df.loc[(df["OS Ecosystem"]=="Windows") & (~df.CLASS.str.contains("Surface")), "CLASS"] = "Windows OEM"
# df.loc[df.CLASS.str.contains("Chromebook"), "CLASS"] = "Chromebook"

In [ ]:
df.loc[(df.Country!="United Kingdom") & (df["OS Ecosystem"]=="Windows") & (df["Sub Family"].str.contains("Surface")), "CLASS"] = df["Sub Family"]
df.loc[(df.Country!="United Kingdom") & (df["OS Ecosystem"]=="Windows") & (~df["Sub Family"].str.contains("Surface")), "CLASS"] = "Windows OEM"
df.loc[(df.Country!="United Kingdom") & (df["OS Ecosystem"]=="Apple"), "CLASS"] = df["Sub Family"]
df.loc[(df.Country!="United Kingdom") & (df["OS Ecosystem"]=="Google") & df["Sub Family"].str.contains("Chromebook"), "CLASS"] = "Chromebook"
df.loc[(df.Country!="United Kingdom") & (df["OS Ecosystem"]=="Google") & (~df["Sub Family"].str.contains("Chromebook", na=False)), "CLASS"] = "Other"
df.loc[(df.Country=="United Kingdom") & (df["OS Ecosystem"]=="Windows") & (df["Sub Family"]=="Undefined"), "CLASS"] = "Windows OEM"
df.loc[(df.Country=="United Kingdom") & (df["OS Ecosystem"]!="Windows") & (df["Sub Family"]=="Undefined"), "CLASS"] = "Other"

In [ ]:
figs = df[df.Country!="United Kingdom"]
uk = df[df.Country=="United Kingdom"]
uk_apple_surface = uk[uk["Sub Family"].isin(["Apple Other", "Surface Pro", "Surface Laptop"])]
uk_other = uk[~uk["Sub Family"].isin(["Apple Other", "Surface Pro", "Surface Laptop"])]

In [ ]:
lookup = pd.read_csv("./lookup/UK Lookup Table.txt")
uk_apple_surface = uk_apple_surface.merge(right=lookup, how="left", on=["Series", "Screen Size Numeric", "PCHeight(mm)"], suffixes=["_old", None])
uk_apple_surface.drop(columns="CLASS_old", inplace=True)

In [ ]:
df = pd.concat([figs, uk_apple_surface, uk_other])

In [ ]:
lookup = pd.read_csv("./lookup/Date Table.txt")
df["Week End"] = pd.to_datetime(df["Week End"], format="%m/%d/%Y")
lookup["Week End"] = pd.to_datetime(lookup["Week End"], format="%d/%m/%Y %H:%M:%S")
df = df.merge(right=lookup, how="left", on="Week End")

In [ ]:
form_factor_map = {
    "Detachable":           "2in1",
    "Traditional Desktop":  "Desktop",
    "Traditional Notebook": "Notebook",
    "Convertible":          "2in1",
    "Large Slate":          "2in1",
    "AIO":                  "Desktop",
    "Ultraslim Notebook":   "Notebook"
}
df.loc[df.CLASS=="Surface Laptop Studio", "Sub Form Factor"] = "Convertible"
df["Form Factor"] = df["Sub Form Factor"].map(form_factor_map)

In [104]:
import xlwings as xw

for fp in os.listdir():
    if re.fullmatch(r"GfK-Weekly-UFIGS-FY\d{2}-FYWK\d{2} CYWK\d{2}.xlsx", fp):
        break

fy = df.FY.max()
fywk = df[df.FY==fy].FYWK.max()
cywk = (fywk + 26) % 52

df = df[[
    "Country", "Sub Family", "Series", "Model Description", "Storage Capacity", "CPU Group",
    "CPU Ram", "Screen Size Numeric", "PCHeight(mm)", "OS Ecosystem", "Sub Form Factor",
    "Price Band USD 3", "Week End", "SALES UNITS", "SALES VALUE USD", "CLASS", "Form Factor",
    "FYWK", "FY"
]]
df = df[df["Week End"] >= "2022-01-08"]

wb = xw.Book(fp)
sheet = wb.sheets("Output_Table")
sheet.range("A:S").clear_contents()
sheet["A1"].options(index=False, header=True).value = df
wb.save(f"GfK-Weekly-UFIGS-FY{fy:01}-FYWK{fywk:01} CYWK{cywk:01}.xlsx")
# wb.close()

KeyboardInterrupt: 